In [1]:
!pip install tslearn


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.4/374.4 kB 3.5 MB/s eta 0:00:00


In [ ]:
import os
import math
import pandas as pd
#import matplotlib.pyplot as plt
import numpy as np
import matplotlib.pyplot as plt
from tslearn.clustering import TimeSeriesKMeans
from sklearn.preprocessing import MinMaxScaler


In [ ]:
directory = 'crypto_data'
clusters_number = 4 # sqrt of 16 different cryptocurrency historic data

In [ ]:
df_BTC = pd.read_csv(directory+ '/BCH-USD_Historical_Data.csv')
df_BTC.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2024-03-27,480.107361,551.345398,479.421234,539.793335,539.793335,1460761937
1,2024-03-28,539.795349,585.368042,528.485901,569.156982,569.156982,1200994117
2,2024-03-29,569.156982,637.706055,556.414246,621.782715,621.782715,1535887261
3,2024-03-30,621.782715,622.934143,591.582642,597.110107,597.110107,648865139
4,2024-03-31,597.116760,691.443604,590.624207,680.421326,680.421326,1008114145


In [ ]:
df_BTC.isnull().sum()

Date         0
Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

In [ ]:
df_BTC.columns

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32 entries, 0 to 31
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       32 non-null     object 
 1   Open       32 non-null     float64
 2   High       32 non-null     float64
 3   Low        32 non-null     float64
 4   Close      32 non-null     float64
 5   Adj Close  32 non-null     float64
 6   Volume     32 non-null     int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 1.9+ KB


In [ ]:
df_BTC.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32 entries, 0 to 31
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       32 non-null     object 
 1   Open       32 non-null     float64
 2   High       32 non-null     float64
 3   Low        32 non-null     float64
 4   Close      32 non-null     float64
 5   Adj Close  32 non-null     float64
 6   Volume     32 non-null     int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 1.9+ KB


In [ ]:
mySeries = []
namesofMySeries = []
for filename in os.listdir(directory):
    if filename.endswith(".csv"):
        df = pd.read_csv(directory + '/' + filename)
        open_values = np.array(df[["Open"]])
        close_values = np.array(df[['Close']])
        change_values = open_values-close_values #calculate the change in price
        df.insert(1,"Change",change_values,True) # add Change data to df
        df = df.loc[:,["Date","Change"]]
        df.set_index("Date",inplace=True) #set date as index
        df.sort_index(inplace=True)

        mySeries.append(df) # add crypto data to list
        namesofMySeries.append(filename.split('_')[0]) #add crypto ticker from start of filename to list

print(namesofMySeries)
print(len(namesofMySeries))

['ETH-USD', 'SHIB-USD', 'BNB-USD', 'BTC-USD', 'TON-USD', 'BCH-USD', 'LINK-USD', 'ADA-USD', 'DOT-USD', 'USDT-USD', 'USDC-USD', 'TRX-USD', 'DOGE-USD', 'SOL-USD', 'AVAX-USD']
15


In [ ]:
print(len(namesofMySeries))
series_lengths = {len(series) for series in mySeries}
print(series_lengths)

15
{32}


In [ ]:
for i in range(len(mySeries)):
    scaler = MinMaxScaler()
    mySeries[i] = MinMaxScaler().fit_transform(mySeries[i])
    mySeries[i]= mySeries[i].reshape(len(mySeries[i]))


print("max: "+str(max(mySeries[0]))+"\tmin: "+str(min(mySeries[0])))
print(mySeries[0][:5])

max: 1.0	min: 0.0
[0.65264769 0.35850041 0.5772782  0.48736892 0.20212539]


In [ ]:
km = TimeSeriesKMeans(n_clusters=clusters_number, metric="dtw")
labels = km.fit_predict(mySeries)
fancy_names_for_labels = [f"Cluster {label}" for label in labels]
results=pd.DataFrame(zip(namesofMySeries,fancy_names_for_labels),columns=["Series","Cluster"]).sort_values(by="Cluster").set_index("Series")
print(results)

            Cluster
Series             
SHIB-USD  Cluster 0
BNB-USD   Cluster 0
TON-USD   Cluster 0
LINK-USD  Cluster 0
ADA-USD   Cluster 0
DOT-USD   Cluster 0
USDT-USD  Cluster 0
USDC-USD  Cluster 0
TRX-USD   Cluster 0
DOGE-USD  Cluster 0
SOL-USD   Cluster 0
AVAX-USD  Cluster 0
BTC-USD   Cluster 1
ETH-USD   Cluster 2
BCH-USD   Cluster 3
